# Grupo 6
## Integrantes 
- Pablo Sarzosa
- Elisa Herrera
- Doménica Rodríguez 
- Andrés Albarracín

# 6.4. Varios canales de entrada y varios canales de salida
 
Si bien hemos descrito los múltiples canales que componen cada imagen (por ejemplo, las imágenes en color tienen los canales RGB estándar para indicar la cantidad de rojo, verde y azul) y las capas convolucionales para múltiples canales en la Sección 6.1.4.1, hasta ahora hemos simplificado todo. de nuestros ejemplos numéricos trabajando con una sola entrada y un solo canal de salida. Esto nos ha permitido pensar en nuestras entradas, núcleos de convolución y salidas, cada uno como tensores bidimensionales.

Cuando agregamos canales a la mezcla, nuestras entradas y representaciones ocultas se convierten en tensores tridimensionales. Por ejemplo, cada imagen de entrada RGB tiene una forma de 3×h×w. Nos referimos a este eje, con un tamaño de 3, como la dimensión del canal. En esta sección, profundizaremos en los núcleos de convolución con múltiples canales de entrada y salida.

## 6.4.1. Múltiples canales de entrada
Cuando los datos de entrada contienen múltiples canales, necesitamos construir un núcleo de convolución con el mismo número de canales de entrada que los datos de entrada, para que pueda realizar una correlación cruzada con los datos de entrada. Suponiendo que el número de canales para los datos de entrada es ci, el número de canales de entrada del kernel de convolución también debe ser ci. Si la forma de la ventana de nuestro núcleo de convolución es kh×kw, entonces cuando ci=1, podemos pensar en nuestro núcleo de convolución como un tensor bidimensional de forma kh×kw.

Sin embargo, cuando ci>1, necesitamos un kernel que contenga un tensor de forma kh×kw para cada canal de entrada. La concatenación de estos tensores ci produce un núcleo de convolución de forma ci×kh×kw. Dado que tanto la entrada como el núcleo de convolución tienen canales ci, podemos realizar una operación de correlación cruzada en el tensor bidimensional de la entrada y el tensor bidimensional del núcleo de convolución para cada canal, sumando los resultados de ci (sumando los canales) para producir un tensor bidimensional. Este es el resultado de una correlación cruzada bidimensional entre una entrada multicanal y un núcleo de convolución multicanal de entrada.

En la figura 6.4.1, mostramos un ejemplo de una correlación cruzada bidimensional con dos canales de entrada. Las partes sombreadas son el primer elemento de salida, así como los elementos de tensor kernel y de entrada utilizados para el cálculo de salida: (1×1+2×2+4×3+5×4)+(0×0+1×1+ 3×2+4×3)=56 .

../_images/conv-multi-in.svg
Fig. 6.4.1 Cálculo de correlación cruzada con 2 canales de entrada.

Para asegurarnos de que realmente entendemos lo que está pasando aquí, podemos implementar nosotros mismos operaciones de correlación cruzada con múltiples canales de entrada. Observe que todo lo que estamos haciendo es realizar una operación de correlación cruzada por canal y luego sumar los resultados.

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use Data::Dumper qw(Dumper);
use AI::MXNet qw(mx);
use List::Util qw(max min shuffle);
#use PDL qw(pdl);
use d2l;
use d2l::Timer;
use d2l::Animator;
use d2l::Accumulator;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly::Plot;
use constant is_train =>'False';

In [2]:
use PDL;

sub corr2d_multi_in {
    my ($X, $K) = @_;
    my $sum = 0;
    for (0 .. $#$X) {
        my $x = $X->[$_];
        my $k = $K->[$_];
        # Implementación de la función corr2d aquí
        $sum += ...;
    }
    return $sum;
}


Warning: Prototype mismatch: sub main::max (@) vs none at (eval 1644) line 7.

Prototype mismatch: sub main::min (@) vs none at (eval 1644) line 7.


Error: syntax error at reply input line 10, near "+= ..."
BEGIN not safe after errors--compilation aborted at reply input line 18.



1

Podemos construir el tensor de entrada X y el tensor kernel K correspondientes a los valores de la figura 6.4.1 para validar la salida de la operación de correlación cruzada.

In [ ]:
use PDL;

my $X = pdl([    [ [0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0] ],
    [ [1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0] ]
]);
my $K = pdl([    [ [0.0, 1.0], [2.0, 3.0] ],
    [ [1.0, 2.0], [3.0, 4.0] ]
]);

corr2d_multi_in($X, $K);


array([[ 56.,  72.],
       [104., 120.]])

## 6.4.2. Múltiples canales de salida
Independientemente de la cantidad de canales de entrada, hasta ahora siempre terminamos con un canal de salida. Sin embargo, como discutimos en la Sección 6.1.4.1, resulta esencial tener múltiples canales en cada capa. En las arquitecturas de redes neuronales más populares, en realidad aumentamos la dimensión del canal a medida que avanzamos en la red neuronal, por lo general, reducimos el muestreo para compensar la resolución espacial por una mayor profundidad del canal. Intuitivamente, podría pensar que cada canal responde a un conjunto diferente de funciones. La realidad es un poco más complicada que las interpretaciones más ingenuas de esta intuición, ya que las representaciones no se aprenden de forma independiente sino que se optimizan para ser útiles en conjunto. Por lo tanto, puede que no sea que un solo canal aprenda un detector de bordes, sino que alguna dirección en el espacio del canal corresponda a la detección de bordes.

Denote por ci y co el número de canales de entrada y salida, respectivamente, y sean kh y kw la altura y el ancho del kernel. Para obtener una salida con múltiples canales, podemos crear un tensor kernel de forma ci×kh×kw para cada canal de salida. Los concatenamos en la dimensión del canal de salida, de modo que la forma del kernel de convolución sea co×ci×kh×kw. En las operaciones de correlación cruzada, el resultado de cada canal de salida se calcula a partir del kernel de convolución correspondiente a ese canal de salida y toma la entrada de todos los canales en el tensor de entrada.

Implementamos una función de correlación cruzada para calcular la salida de múltiples canales como se muestra a continuación.

In [ ]:
use PDL;

sub corr2d_multi_in_out {
    my ($X, $K) = @_;
    my @results;
    for my $k (@$K) {
        push @results, corr2d_multi_in($X, $k);
    }
    return cat(@results);
}


Construimos un kernel de convolución con 3 canales de salida concatenando el tensor del kernel K con K+1 (más uno para cada elemento en K) y K+2.

In [ ]:
use PDL;

my $K_stacked = cat(map { $K + $_ } 0 .. 2);
my @shape = dims($K_stacked);


(3, 2, 2, 2)

A continuación, realizamos operaciones de correlación cruzada en el tensor de entrada X con el tensor kernel K. Ahora la salida contiene 3 canales. El resultado del primer canal es consistente con el resultado del tensor de entrada anterior X y el kernel de canal de entrada única y canal de entrada múltiple.

In [ ]:
print corr2d_multi_in_out($X, $K);


array([[[ 56.,  72.],
        [104., 120.]],

       [[ 76., 100.],
        [148., 172.]],

       [[ 96., 128.],
        [192., 224.]]])

## 6.4.3. Capa convolucional 1×1
Al principio, una convolución de 1×1, es decir, kh=kw=1, no parece tener mucho sentido. Después de todo, una convolución correlaciona píxeles adyacentes. Una convolución 1×1 obviamente no lo hace. No obstante, son operaciones populares que a veces se incluyen en los diseños de redes profundas complejas. Veamos con cierto detalle lo que realmente hace.

Debido a que se usa la ventana mínima, la convolución 1×1 pierde la capacidad de las capas convolucionales más grandes para reconocer patrones que consisten en interacciones entre elementos adyacentes en las dimensiones de altura y anchura. El único cálculo de la convolución 1×1 ocurre en la dimensión del canal.

La figura 6.4.2 muestra el cálculo de la correlación cruzada utilizando el kernel de convolución 1×1 con 3 canales de entrada y 2 canales de salida. Tenga en cuenta que las entradas y salidas tienen la misma altura y anchura. Cada elemento de la salida se deriva de una combinación lineal de elementos en la misma posición en la imagen de entrada. Podría pensar en la capa convolucional 1 × 1 como una capa completamente conectada aplicada en cada ubicación de píxel para transformar los valores de entrada correspondientes ci en valores de salida co. Debido a que esta sigue siendo una capa convolucional, los pesos están vinculados a la ubicación de los píxeles. Por lo tanto, la capa convolucional 1 × 1 requiere pesos co × ci (más el sesgo).

Verifiquemos si esto funciona en la práctica: implementamos una convolución 1×1 usando una capa completamente conectada. Lo único es que necesitamos hacer algunos ajustes a la forma de los datos antes y después de la multiplicación de matrices.

In [ ]:
use PDL;

sub corr2d_multi_in_out_1x1 {
    my ($X, $K) = @_;
    my ($c_i, $h, $w) = dims($X);
    my ($c_o) = dims($K, 0);
    $X = $X->reshape($c_i, $h * $w);
    $K = $K->reshape($c_o, $c_i);
    my $Y = $K x $X;
    return $Y->reshape($c_o, $h, $w);
}


1

Verifiquemos si esto funciona en la práctica: implementamos una convolución 1×1 usando una capa completamente conectada. Lo único es que necesitamos hacer algunos ajustes a la forma de los datos antes y después de la multiplicación de matrices.

In [ ]:
use PDL;

my $X = random(3, 3, 3) * 2 - 1;
my $K = random(2, 3, 1, 1) * 2 - 1;

my $Y1 = corr2d_multi_in_out_1x1($X, $K);
my $Y2 = corr2d_multi_in_out($X, $K);

if (max(abs($Y1 - $Y2)) > 1e-6) {
    die "Assertion failed";
}


1

## 6.4.4. Resumen
Se pueden usar múltiples canales para extender los parámetros del modelo de la capa convolucional.

La capa convolucional 1×1 es equivalente a la capa totalmente conectada, cuando se aplica por píxel.

La capa convolucional 1×1 se usa típicamente para ajustar la cantidad de canales entre las capas de la red y para controlar la complejidad del modelo.